## 08데이터 전처리 실습-1
### 파이썬 클래스 설정
#### 2020/06/07 keonwoo Park

In [13]:
class Calculator:
    def __init__(self):   # 생성자 함수. 실제 객체(인스턴스)하나 만듬
        self.result = 0
    
    def add(self, num):
        self.result += num
        return self.result

In [24]:
cal1 = Calculator()
cal2 = Calculator()

print(cal1.add(3))
print(cal1.add(4))
print(cal2.add(3))
print(cal2.add(7))

3
7
3
10


### text파일로 함수를지정해두면 import를 통해서 불러올 수 있다.
#### 충격..

In [20]:
import mod1

In [22]:
mod1.add(5,7)

12

In [25]:
mod1.sub(20, 30)

-10

## 데이터 정제(Data cleansing)
#### 결측값
- 데이터 개체 또는 속성의 제거
- 수동으로 입력
- 전역상수 입력
- 결측값 무시
- 결측값의 추정 (전체 평균값, 같은 클래스의 평균값, 가장 가능성이 높은 값)

#### 잡음 제거
- 구간화
- 회귀
- 군집화

## 결측값 처리(1)

In [26]:
import pandas as pd
import numpy as np

In [29]:
data = pd.read_csv("약수터수질현황.csv")

In [35]:
data_1 = data.set_index("연번")

In [68]:
data_1.head()

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
1,백수,모라,양 성,10.0,6.7,부적합
2,이칠,모라,음 성,20.0,0.9,적합
3,운수사,모라,음 성,10.0,1.1,적합
4,서당골,모라,음 성,10.0,NaN,적합
5,청수,괘법,음 성,20.0,2.7,적합


In [42]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 1 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   약수터명    24 non-null     object 
 1   동명      24 non-null     object 
 2   총대장균군   24 non-null     object 
 3   일반세균    22 non-null     float64
 4   질산성질소   22 non-null     float64
 5   적합      24 non-null     object 
dtypes: float64(2), object(4)
memory usage: 1.3+ KB


### 결측값을 갖는 튜플을 무시한다.(해당 행을 제거하는 방법)

In [69]:
missing_data = data_1.dropna() # .dropba() 널값을 갖는 행을 제거한다.
missing_data.head()

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
1,백수,모라,양 성,10.0,6.7,부적합
2,이칠,모라,음 성,20.0,0.9,적합
3,운수사,모라,음 성,10.0,1.1,적합
5,청수,괘법,음 성,20.0,2.7,적합
6,사상,괘법,음 성,10.0,2.2,적합


### 전역상수를 사용하여 결측값을 채워 넣는다.
### 모든 널값을 하나의 값으로 통일하는 것

In [56]:
fill_constant_data = data_1.fillna(0)

In [59]:
fill_constant_data.query('연번 in (4,17,20)') # 기존 널값이 존재하던 행들

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
4,서당골,모라,음 성,10.0,0.0,적합
17,밤골,주례,음 성,0.0,0.0,적합
20,승학,학장,양 성,0.0,0.3,부적합


### 속성의 평균값을 사용하여 결측값을 채워 넣기

In [61]:
fill_mean_data = data_1.fillna(data_1.mean())
fill_mean_data.query("연번 in (4, 17, 20)")

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
4,서당골,모라,음 성,10.000000,2.631818,적합
17,밤골,주례,음 성,20.454545,2.631818,적합
20,승학,학장,양 성,20.454545,0.300000,부적합


### 주어진 튜플과 같은 클래스(분류)에 속하는 튜플들의 속성의 평균

#### 열의값을 평균으로만 채워 넣는게아니라.
#### 데이터를 분류하여 각각에 맞는 평균값을 넣는 것
- "일반세균" 전체의 평균이 아닌 일반세균 중의 적합과 부적합 값의 각각의 평균에 맞춰서 집어넣는다.
- 일반세균의 널값이 적합이다 -> 적합값들의 평균만 넣어둔다.
- 일반세균의 널값이 부적합이다 -> 부적합값들의 평균만 넣어둔다.
- groupby('적합')은 적합이라는 열을 그룹으로 나눈것이다.(적합, 부적합 두 개로)

In [66]:
copy_data = pd.DataFrame(data_1, copy=True)
copy_data['일반세균'].fillna(data_1.groupby("적합")['일반세균'].transform('mean'),inplace=True)
copy_data['질산성질소'].fillna(data_1.groupby("적합")['질산성질소'].transform('mean'),inplace=True)

In [67]:
copy_data.query("연번 in (4, 17, 20)")

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
4,서당골,모라,음 성,10.000000,2.706667,적합
17,밤골,주례,음 성,15.625000,2.706667,적합
20,승학,학장,양 성,33.333333,0.300000,부적합


### 가장 가능성이 높은 값으로 결측값을 채워 넣기
- 데이터간의 상관관계를 통해 결측값을 채워 넣는다.

In [71]:
from scipy import stats # 선형회귀분석을 위한 패키지

In [72]:
# 일반세균과 질산성질소 간의 회귀분석

reg_data = pd.DataFrame(data_1, copy=True)

In [91]:
# 널값을 제외하고 선형회귀 분석 수행
mask = ~np.isnan(reg_data['일반세균']) & ~np.isnan(reg_data['질산성질소'])

In [97]:
# 질산성질소가 종속변수일때 회귀분석
slope, intercept, r_value, p_value, std_err = stats.linregress(
reg_data['일반세균'][mask], reg_data['질산성질소'][mask])

In [84]:
reg_data.loc[4,'질산성질소']

nan

In [87]:
reg_data.loc[4, '질산성질소'] = reg_data['일반세균'][4]*slope + intercept

In [98]:
# NaN(null) 값을 제외하고 선형회귀 분석 수행
# 일반세균이 종속변수 일때 회귀분석
slope, intercept, r_value, p_value, std_err = stats.linregress(
reg_data['질산성질소'][mask], reg_data['일반세균'][mask])

In [95]:
# 질산성질소 값에 의한 일반세균값 산출
reg_data.loc[20,'일반세균'] = reg_data['질산성질소'][20]*slope + intercept

In [106]:
reg_data.query('연번 in (4,20)')

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
4,서당골,모라,음 성,10.000000,3.045076,적합
20,승학,학장,양 성,31.556376,0.300000,부적합


In [107]:
reg_data

,약수터명,동명,총대장균군,일반세균,질산성질소,적합
연번,,,,,,
1,백수,모라,양 성,10.000000,6.700000,부적합
2,이칠,모라,음 성,20.000000,0.900000,적합
3,운수사,모라,음 성,10.000000,1.100000,적합
4,서당골,모라,음 성,10.000000,3.045076,적합
5,청수,괘법,음 성,20.000000,2.700000,적합
6,사상,괘법,음 성,10.000000,2.200000,적합
7,탑골,괘법,양 성,10.000000,2.600000,부적합
8,삼각산,괘법,음 성,10.000000,2.700000,적합
9,괘내,괘법,음 성,20.000000,3.100000,적합
